In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((4120, 13), (4120, 13))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove the column that specifies which data fold we used
df = df.drop(['fold', 'AL', 'PL', 'vit', 'model', 'data', 'ALL', 'method', 'ratio', 'kw'], axis = 1)

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,dataset,metric,value,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,3.84,5,0.0005,64
1,UCM-CLS,zeroshot-val-top1,7.14,5,0.0005,64
2,RSICD,image_to_text_R@1,0.18,5,0.0005,64
3,RSICD,image_to_text_R@5,0.82,5,0.0005,64
4,RSICD,image_to_text_R@10,1.46,5,0.0005,64
...,...,...,...,...,...,...
4115,Sydney,image_to_text_R@5,3.22,35,0.0005,128
4116,Sydney,image_to_text_R@10,5.63,35,0.0005,128
4117,Sydney,text_to_image_R@1,0.60,35,0.0005,128
4118,Sydney,text_to_image_R@5,3.82,35,0.0005,128


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value                 
                                                       mean        std count
lr       bs  metric             dataset   epochs                            
0.000005 64  image_to_text_R@1  RSICD     5        1.665556   0.388269     9
                                          15       3.352222   0.320966     9
                                          25       4.925556   0.351679     9
                                          35       5.544444   0.666260     9
                                Sydney    5        2.347778   0.543318     9
                                          15       3.085556   0.426823     9
                                          25       3.331111   0.401386     9
                                          35       3.352222   0.550313     9
                                UCM       5        5.822222   1.233185     9
                                          15       8.571111   0.951255     9
                                          25       9.946667   1.998862     9
                                          35      10.846667   1.731401     9
             image_to_text_R@10 RSICD     5       14.514444   1.001301     9
                                          15      25.136667   1.526499     9
                                          25      27.218889   1.464953     9
                                          35      28.560000   1.342824     9
                                Sydney    5       14.421111   1.155297     9
                                          15      18.688889   0.850903     9
                                          25      18.913333   0.732137     9
                                          35      19.695556   0.924596     9
                                UCM       5       52.114444   3.481663     9
                                          15      69.682222   3.401690     9
                                          25      73.385556   2.397113     9
                                          35      74.974444   3.051623     9
             image_to_text_R@5  RSICD     5        7.708889   0.921038     9
                                          15      14.340000   0.709049     9
                                          25      16.615556   1.140429     9
                                          35      17.427778   0.701940     9
                                Sydney    5        8.203333   0.672737     9
                                          15      11.713333   0.791470     9
                                          25      11.692222   0.903601     9
                                          35      12.072222   0.754350     9
                                UCM       5       30.582222   2.562629     9
                                          15      42.010000   2.561338     9
                                          25      45.662222   2.535019     9
                                          35      45.554444   2.297973     9
             text_to_image_R@1  RSICD     5        1.320000   0.289353     9
                                          15       3.025556   0.439548     9
                                          25       4.652222   0.747860     9
                                          35       5.534444   0.328295     9
                                Sydney    5        2.324444   0.416897     9
                                          15       2.772222   0.637177     9
                                          25       3.310000   0.541664     9
                                          35       3.600000   0.582301     9
                                UCM       5        6.721111   0.502679     9
                                          15       8.413333   1.325490     9
                                          25      10.318889   1.701576     9
                                          35      10.951111   1.543036     9
             text_to_image_R@10 RSICD     5       12.991111   0.624689     9
                                          15      

In [7]:
df_grouped.to_csv('eval.csv')

In [11]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2[(df_2['epochs'] > 5) & (df_2['lr']==5e-5) & (df_2['bs'] == 64)] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

lr  bs              metric    dataset epochs      value            \
                                                             mean       std   
161  0.00005  64   image_to_text_R@1      RSICD     15   7.058889  0.648237   
162  0.00005  64   image_to_text_R@1      RSICD     25   7.811111  0.918864   
163  0.00005  64   image_to_text_R@1      RSICD     35   8.723333  0.710774   
165  0.00005  64   image_to_text_R@1     Sydney     15   3.376667  1.004453   
167  0.00005  64   image_to_text_R@1     Sydney     35   4.226667  0.595840   
166  0.00005  64   image_to_text_R@1     Sydney     25   4.360000  0.847068   
169  0.00005  64   image_to_text_R@1        UCM     15  11.322222  2.318975   
170  0.00005  64   image_to_text_R@1        UCM     25  12.326667  2.053655   
171  0.00005  64   image_to_text_R@1        UCM     35  13.121111  2.035684   
173  0.00005  64  image_to_text_R@10      RSICD     15  34.868889  1.566560   
174  0.00005  64  image_to_text_R@10      RSICD     25  35.273333  1.305115   
175  0.00005  64  image_to_text_R@10      RSICD     35  35.771111  1.356682   
177  0.00005  64  image_to_text_R@10     Sydney     15  20.883333  1.648833   
179  0.00005  64  image_to_text_R@10     Sydney     35  21.975556  1.513870   
178  0.00005  64  image_to_text_R@10     Sydney     25  22.580000  2.134696   
181  0.00005  64  image_to_text_R@10        UCM     15  78.676667  3.201972   
182  0.00005  64  image_to_text_R@10        UCM     25  79.523333  4.033984   
183  0.00005  64  image_to_text_R@10        UCM     35  80.264444  3.078364   
185  0.00005  64   image_to_text_R@5      RSICD     15  22.161111  0.726231   
186  0.00005  64   image_to_text_R@5      RSICD     25  22.556667  0.967032   
187  0.00005  64   image_to_text_R@5      RSICD     35  22.751111  1.209953   
189  0.00005  64   image_to_text_R@5     Sydney     15  13.100000  1.543745   
191  0.00005  64   image_to_text_R@5     Sydney     35  14.285556  0.792167   
190  0.00005  64   image_to_text_R@5     Sydney     25  14.420000  1.286313   
193  0.00005  64   image_to_text_R@5        UCM     15  47.567778  2.612546   
194  0.00005  64   image_to_text_R@5        UCM     25  50.053333  4.400949   
195  0.00005  64   image_to_text_R@5        UCM     35  50.476667  3.149579   
197  0.00005  64   text_to_image_R@1      RSICD     15   7.373333  0.619819   
198  0.00005  64   text_to_image_R@1      RSICD     25   8.612222  0.664074   
199  0.00005  64   text_to_image_R@1      RSICD     35   9.424444  0.534488   
201  0.00005  64   text_to_image_R@1     Sydney     15   4.535556  0.713111   
202  0.00005  64   text_to_image_R@1     Sydney     25   4.918889  0.333333   
203  0.00005  64   text_to_image_R@1     Sydney     35   5.544444  0.463468   
205  0.00005  64   text_to_image_R@1        UCM     15  11.641111  1.329196   
206  0.00005  64   text_to_image_R@1        UCM     25  13.120000  1.829713   
207  0.00005  64   text_to_image_R@1        UCM     35  13.863333  0.907538   
209  0.00005  64  text_to_image_R@10      RSICD     15  34.185556  1.094613   
210  0.00005  64  text_to_image_R@10      RSICD     25  34.391111  1.310367   
211  0.00005  64  text_to_image_R@10      RSICD     35  34.663333  1.213528   
213  0.00005  64  text_to_image_R@10     Sydney     15  23.250000  1.830717   
214  0.00005  64  text_to_image_R@10     Sydney     25  24.547778  1.516944   
215  0.00005  64  text_to_image_R@10     Sydney     35  25.328889  1.563685   
218  0.00005  64  text_to_image_R@10        UCM     25  77.990000  2.279995   
217  0.00005  64  text_to_image_R@10        UCM     15  78.095556  2.826474   
219  0.00005  64  text_to_image_R@10        UCM     35  79.101111  2.276013   
221  0.00005  64   text_to_image_R@5      RSICD     15  21.625556  0.977102   
222  0.00005  64   text_to_image_R@5      RSICD     25  22.403333  0.666990   
223  0.00005  64   text_to_image_R@5      RSICD     35  22.853333  0.924040   
225  0.00005  64   text_to_image_R@5     Sydney     15  14.0